<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li></ul></div>

#### Imports

In [1]:
import logging
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
def _get_response(url):
    response = requests.get(url)
    return response

In [4]:
def get_soup_object(url):
    response = _get_response(url)
    #Insert logic to check
    #if response = 200
    soup = BeautifulSoup(response.text, "html.parser")
    #else handle exceptions
    return soup

In [5]:
def get_aircraft_urls(url, aircraft_url_input_dir):
    df_aircraft_urls = pd.read_table(aircraft_url_input_dir)
    aircraft_url_list = []
    for index in range(len(df_aircraft_urls)):
        url_suffix_aircraft = df_aircraft_urls['URL'][index]
        aircraft_url_list.append(f'{url}{url_suffix_aircraft}')
    logging.debug(f'URLs read in successfully.')
    return df_aircraft_urls, aircraft_url_list

In [6]:
def _read_table(table_number, soup):
    table = soup.find_all('table')[table_number]
    rows = table.find_all('tr')
    row_list = list()
    for tr in rows:
        td = tr.find_all('td')
        row = [i.text for i in td]
        row_list.append(row)
    flat_list = [item for sublist in row_list for item in sublist]
    return flat_list

In [7]:
def _get_table_header(table, index_list, header_list):
    
    for i in range(len(table)):
        header = table[i].find_all('th')
        header_string = [i.text for i in header]
        index_list.append(i)
        header_list.append(header_string)
        logging.debug(f"Index: {i}")
        logging.debug(f"Header: {header_string}")
    
    return index_list, header_list

In [8]:
def count_tables_on_page(soup):
    table = soup.find_all('table')
    number_of_tables = len(table)
    logging.debug(f"There are {number_of_tables} tables on this page.")
    index_list = []
    header_list = []
    if len(table) > 0:
        index_list, header_list = _get_table_header(table, index_list, header_list)
    else:
        logging.error(f"No tables found.")
    
    table_header_dict = {"index": index_list,
            "header": header_list}
    
    return number_of_tables, table_header_dict

In [9]:
def read_table_one(soup, table_number):
    
    flat_list = _read_table(table_number, soup)
    
    logging.debug(f"flat_list is {flat_list}")
    
    _list = []
    dict_keys = []
    dict_values = []
    
    flat_list = flat_list[:13] #Specific - change them ???????????
    
    for i in range(len(flat_list)):
        _list_val = flat_list[i]
        _sublist = _list_val.split(': ') #Specific to this context
        _list.append(_sublist)
    
    logging.debug(f"_list: {_list}")
    logging.debug(f"length of _list: {len(_list)}")
    
    for j in range(len(_list)):
        logging.debug(f"Current dict key is: {_list[j][0]}.")
        dict_keys.append(_list[i][0])
        logging.debug(f"Current dict value is: {_list[j][1]}.")
        dict_values.append(_list[i][1])
    
    _dict = dict(zip(dict_keys, dict_values))
    
    df_table_1 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_1

In [10]:
def check_pattern_existence(pattern, string):
    if re.search(pattern, string):
        indicator = True
    else:
        indicator = False
    
    return indicator

In [11]:
def read_table_two(soup, table_number):
    
    flat_list = _read_table(table_number, soup)
    
    _list = []
    dict_keys = []
    dict_values = []
    
    logging.debug(f"Length of flat_list is: {len(flat_list)}.")
    logging.debug(f"flat_list is: {flat_list}.")
    
    next_line_pattern = '\\n[\\t]+'
    max_range_pattern = ' Max Range: '
    
    for i in range(len(flat_list)):
        _list_val = flat_list[i]
        if check_pattern_existence(max_range_pattern, _list_val) and \
        check_pattern_existence(next_line_pattern, _list_val):
            _sublist_2 = _list_val.split(' Max Range: ')
            _sublist = re.split('\\n[\\t]+', _sublist_2[0])
            _sublist.append(_sublist_2[1]) #Specific to this context
            _list.append(_sublist)
        else:
            _list.append([_list_val])
    
    for i in range(len(_list)):
        dict_keys_template = [f'Sensor {i+1} Name', f'Sensor {i+1} Type', f'Sensor {i+1} Max Range']
        for j in range(len(_list[i])):
            dict_keys.append(dict_keys_template[j])
            dict_values.append(_list[i][j])
    
    _dict = dict(zip(dict_keys, dict_values))
    
    df_table_2 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_2

In [12]:
def read_table_three(soup, table_number):
    flat_list = _read_table(table_number, soup)
    
    _list = []
    dict_keys = []
    dict_values = []

    next_line_pattern = '\\n[\\t]+'
    
    for i in range(len(flat_list)):
        logging.debug(f"i:{i}")
        _list_val = flat_list[i]
        _sublist = re.split(next_line_pattern, _list_val)
        _sublist_2 = _sublist[-1].split(". ")
        _sublist_2 = [i for i in _sublist_2 if i]
        _list.append(_sublist)
        _list.append(_sublist_2)

        weapon_keys = []
        weapon_values = []

        dict_keys_template = [f'Weapon {i+1} Name', f'Weapon {i+1} Description']
        logging.debug(f"_list:{_list}")
        logging.debug(f"_sublist_2:{_sublist_2}")
        if len(_sublist_2) > 1:
            for j in range(len(_sublist_2)):
                _sublist_3 = _sublist_2[j].split(":")
                weapon_key = f"Weapon {i+1} {_sublist_3[0]}"
                weapon_value = _sublist_3[1]
                weapon_keys.append(weapon_key)
                weapon_values.append(weapon_value)
                logging.debug(f"weapon_keys: {weapon_keys}")
                logging.debug(f"weapon_values: {weapon_values}")


            row_keys = dict_keys_template + weapon_keys
            row_values = _sublist[:2] + weapon_values

        elif len(_sublist_2) == 1 :
            weapon_key = f"Weapon {i+1}"
            weapon_keys.append(weapon_key)
            weapon_value = _sublist_2[0]
            weapon_values.append(weapon_value)
            logging.debug(f"weapon_keys: {weapon_keys}")
            logging.debug(f"weapon_values: {weapon_values}")

            row_keys = dict_keys_template
            row_values = _list[i]


        dict_keys = dict_keys + row_keys
        dict_values = dict_values + row_values
        logging.debug(f"dict_keys: {dict_keys}")
        logging.debug(f"dict_values: {dict_values}")
    
    _dict = dict(zip(dict_keys, dict_values))

    df_table_3 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_3

In [13]:
def read_image(soup, url):
    images = soup.findAll('img')
    #insert logic to choose correct index of image from images list
    image_url = images[1]['src']
    complete_url = f'{url}{image_url}'
    
    #perhaps insert try catch logic here to cater for exceptions
    image_data = requests.get(complete_url).content
    
    logging.debug(f'Image data recieved from URL.')
    
    return image_data

In [14]:
def save_image(image_data, image_output_dir, image_name, image_type):
    with open(f'{image_output_dir}{image_name}.{image_type}', 'wb') as handler:
        handler.write(img_data)
    logging.debug(f'Image written to file: {image_output_dir}{image_name}.{image_type}')

In [15]:
def combine_tables_into_dataframe(df_to_combine):
#     Something along these lines - but better naming convention:
#     aircraft_data['Name'] = df_aircraft['Link'][0]
#     aircraft_data = pd.concat([df_table_1, df_table_2, df_table_3], axis=1, sort=False)
    aircraft_data = pd.concat(df_to_combine, axis=1, sort=False)
    logging.debug(f"The final df looks like:")
    logging.debug(f"{aircraft_data}")
    return aircraft_data

In [16]:
def write_dataframe_to_file(dataframe, csv_output_dir, csv_output_filename):
    logging.info(f'...File {csv_output_filename}.csv is being written to disk...')
    dataframe.to_csv(f'{csv_output_dir}{csv_output_filename}.csv', index=False)
    logging.info(f'Writing complete.')

In [17]:
def main():
    aircraft_url_input_dir = '../data/01_raw/aircraft_url.txt'
    csv_output_dir = '../data/02_intermediate/'
    csv_output_filename = 'aircraft_data'
    image_output_dir = '../data/01_raw/weapons_images/'
    image_type = 'jpg'
    root_url = 'http://cmano-db.com/'
    
    logging.info(f'Beginning Process.')
    
    df_aircraft_urls, aircraft_url_list = get_aircraft_urls(root_url, aircraft_url_input_dir)
    
    aircraft_url_list = aircraft_url_list[:20]
    
    aircraft_data = pd.DataFrame()
    
    for index in range(len(aircraft_url_list)):
        
        url = aircraft_url_list[index]
        
        logging.debug(f'Current URL being processed is: {url} at index: {index}.')
        soup = get_soup_object(url)
        
        number_of_tables, table_header_dict = count_tables_on_page(soup)
        
        df_to_combine = []
        
        for table_number in range(number_of_tables):
            
            logging.debug(f"table_number is {table_number}")
            logging.debug(f"table_header_dict is {table_header_dict}")
            
            _index = table_header_dict['index'][table_number]
            _header = table_header_dict['header'][table_number][0]
            
            logging.debug(f"_index is {_index}")
            logging.debug(f"_header is {_header}")
            
            if _header == "General data:":      
                df_table_1 = read_table_one(soup, table_number)
                logging.debug(f'Table 1 returned successfully.')
                df_to_combine.append(df_table_1)
            elif _header == "Sensors / EW:":
                df_table_2 = read_table_two(soup, table_number)
                logging.debug(f'Table 2 returned successfully.')
                df_to_combine.append(df_table_2)
            elif _header == "Weapons / Loadouts:":
                df_table_3 = read_table_three(soup, table_number)
                logging.debug(f'Table 3 returned successfully.')
                df_to_combine.append(df_table_3)
            _aircraft_data = combine_tables_into_dataframe(df_to_combine)
            logging.debug(f'Table 1, 2 and 3 combined successfully.')       
        aircraft_data = pd.concat([aircraft_data, _aircraft_data])
        logging.debug(f'Row {index} appended to dataframe.')
  
    write_dataframe_to_file(aircraft_data, csv_output_dir, csv_output_filename)

    logging.info(f'Process Complete.')

In [18]:
if __name__ == '__main__':
    main()

INFO:root:Beginning Process.
DEBUG:root:URLs read in successfully.
DEBUG:root:Current URL being processed is: http://cmano-db.com/aircraft/4354/ at index: 0.
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): cmano-db.com:80
DEBUG:urllib3.connectionpool:http://cmano-db.com:80 "GET /aircraft/4354/ HTTP/1.1" 200 None
DEBUG:root:There are 3 tables on this page.
DEBUG:root:Index: 0
DEBUG:root:Header: ['General data:', '']
DEBUG:root:Index: 1
DEBUG:root:Header: ['Sensors / EW:']
DEBUG:root:Index: 2
DEBUG:root:Header: ['Weapons / Loadouts:']
DEBUG:root:table_number is 0
DEBUG:root:table_header_dict is {'index': [0, 1, 2], 'header': [['General data:', ''], ['Sensors / EW:'], ['Weapons / Loadouts:']]}
DEBUG:root:_index is 0
DEBUG:root:_header is General data:
DEBUG:root:flat_list is ['Type: Attack', 'Crew: 2', 'Min Speed: 220 kt', 'Max Speed: 330 kt', 'Wingspan: 11.1 m', 'Height: 3.9 m', 'Length: 11.4 m', 'Max Payload: 1500 kg', 'Empty Weight: 3200 kg', 'Max Weight: 5400 kg', 'Oper

DEBUG:urllib3.connectionpool:http://cmano-db.com:80 "GET /aircraft/3040/ HTTP/1.1" 200 None
DEBUG:root:There are 3 tables on this page.
DEBUG:root:Index: 0
DEBUG:root:Header: ['General data:', '']
DEBUG:root:Index: 1
DEBUG:root:Header: ['Sensors / EW:']
DEBUG:root:Index: 2
DEBUG:root:Header: ['Weapons / Loadouts:']
DEBUG:root:table_number is 0
DEBUG:root:table_header_dict is {'index': [0, 1, 2], 'header': [['General data:', ''], ['Sensors / EW:'], ['Weapons / Loadouts:']]}
DEBUG:root:_index is 0
DEBUG:root:_header is General data:
DEBUG:root:flat_list is ['Type: Fighter', 'Crew: 1', 'Min Speed: 350 kt', 'Max Speed: 920 kt', 'Wingspan: 7.2 m', 'Height: 4.1 m', 'Length: 14.0 m', 'Max Payload: 0 kg', 'Empty Weight: 4871 kg', 'Max Weight: 8625 kg', 'Operator: Air Force', 'Commissioned: 1980', 'Propulsion: 1x WP-7A', '\n']
DEBUG:root:_list: [['Type', 'Fighter'], ['Crew', '1'], ['Min Speed', '350 kt'], ['Max Speed', '920 kt'], ['Wingspan', '7.2 m'], ['Height', '4.1 m'], ['Length', '14.0 m'],

DEBUG:root:i:5
DEBUG:root:_list:[['480 liter Drop Tank -', ' Drop Tank\n'], [' Drop Tank\n'], ['250kg GPB -', '(Generic) Bomb', 'Surface Max: 1.9 km. Land Max: 1.9 km. '], ['Surface Max: 1.9 km', 'Land Max: 1.9 km'], ['500kg GPB -', '(Generic) Bomb', 'Surface Max: 1.9 km. Land Max: 1.9 km. '], ['Surface Max: 1.9 km', 'Land Max: 1.9 km'], ['57mm Rocket -', '(Generic) Rocket', 'Surface Max: 1.9 km. Land Max: 1.9 km. '], ['Surface Max: 1.9 km', 'Land Max: 1.9 km'], ['90mm Rocket -', '(Generic) Rocket', 'Surface Max: 3.7 km. Land Max: 3.7 km. '], ['Surface Max: 3.7 km', 'Land Max: 3.7 km'], [' PL-2a [AA-2a] -', '(1971) Guided Weapon', 'Air Max: 9.3 km. '], ['Air Max: 9.3 km']]
DEBUG:root:_sublist_2:['Air Max: 9.3 km']
DEBUG:root:weapon_keys: ['Weapon 6']
DEBUG:root:weapon_values: ['Air Max: 9.3 km']
DEBUG:root:dict_keys: ['Weapon 1 Name', 'Weapon 1 Description', 'Weapon 2 Name', 'Weapon 2 Description', 'Weapon 2 Surface Max', 'Weapon 2 Land Max', 'Weapon 3 Name', 'Weapon 3 Description', 'W

DEBUG:root:dict_values: ['Cargo [No Drop, 10 tons] -', ' Cargo\n', ' Cargo\n', 'Cargo [No Drop, 20 tons] -', ' Cargo\n']
DEBUG:root:i:3
DEBUG:root:_list:[['Cargo [No Drop, 10 tons] -', ' Cargo\n'], [' Cargo\n'], ['Cargo [No Drop, 20 tons] -', ' Cargo\n'], [' Cargo\n'], ['Cargo [Air Drop, 7 tons] -', ' Cargo\n'], [' Cargo\n'], ['Paratrooper -', ' Paratroops\n'], [' Paratroops\n']]
DEBUG:root:_sublist_2:[' Paratroops\n']
DEBUG:root:weapon_keys: ['Weapon 4']
DEBUG:root:weapon_values: [' Paratroops\n']
DEBUG:root:dict_keys: ['Weapon 1 Name', 'Weapon 1 Description', 'Weapon 2 Name', 'Weapon 2 Description', 'Weapon 3 Name', 'Weapon 3 Description', 'Weapon 4 Name', 'Weapon 4 Description']
DEBUG:root:dict_values: ['Cargo [No Drop, 10 tons] -', ' Cargo\n', ' Cargo\n', 'Cargo [No Drop, 20 tons] -', ' Cargo\n', ' Cargo\n']
DEBUG:root:i:4
DEBUG:root:_list:[['Cargo [No Drop, 10 tons] -', ' Cargo\n'], [' Cargo\n'], ['Cargo [No Drop, 20 tons] -', ' Cargo\n'], [' Cargo\n'], ['Cargo [Air Drop, 7 tons] 

DEBUG:root:dict_values: ['AR-1 Air Ranger ATGM -', '(-China, HJ-10 Mod, [She Shou]) Guided Weapon', ' 9.3 km', ' 9.3 km', 'Surface Max: 9.3 km', 'Land Max: 9.3 km']
DEBUG:root:Table 3 returned successfully.
DEBUG:root:The final df looks like:
DEBUG:root:  Propulsion Sensor 1 Max Range  \
0  1x Piston            83.3 km   

                                     Sensor 1 Name  \
0  Generic FLIR - (3rd Gen, Surveillance) Infrared   

                   Sensor 1 Type Sensor 2 Max Range  \
0  Infrared, Surveillance Camera            55.6 km   

                                       Sensor 2 Name  \
0  Generic TV Camera - (3rd Gen, Surveillance) Vi...   

                    Sensor 2 Type Sensor 3 Max Range  \
0  Visual, Surveillance TV Camera            18.5 km   

                                       Sensor 3 Name  \
0  Generic Laser Designator - (Surface Only) Lase...   

                              Sensor 3 Type  \
0  Laser Target Designator & Ranger (LTD/R)   

                     

...

DEBUG:root:Table 1 returned successfully.
DEBUG:root:The final df looks like:
DEBUG:root:      Propulsion
0  2x R-15BD-300
DEBUG:root:Table 1, 2 and 3 combined successfully.
DEBUG:root:table_number is 1
DEBUG:root:table_header_dict is {'index': [0, 1, 2], 'header': [['General data:', ''], ['Sensors / EW:'], ['Weapons / Loadouts:']]}
DEBUG:root:_index is 1
DEBUG:root:_header is Sensors / EW:
DEBUG:root:Length of flat_list is: 2.
DEBUG:root:flat_list is: ['SPO-3 Sirena-3 -  ESM\n\t\t\t\t\tRWR, Radar Warning Receiver Max Range: 222.2 km', 'Fox Fire [RP-SA Smerch-A3] - (MiG-25P) Radar\n\t\t\t\t\tRadar, FCR, Air-to-Air, Medium-Range Max Range: 111.1 km'].
DEBUG:root:Table 2 returned successfully.
DEBUG:root:The final df looks like:
DEBUG:root:      Propulsion Sensor 1 Max Range          Sensor 1 Name  \
0  2x R-15BD-300           222.2 km  SPO-3 Sirena-3 -  ESM   

                 Sensor 1 Type Sensor 2 Max Range  \
0  RWR, Radar Warning Receiver           111.1 km   

                    

DEBUG:root:The final df looks like:
DEBUG:root:      Propulsion Sensor 1 Max Range  \
0  2x R-15BD-300            74.1 km   

                                Sensor 1 Name  \
0  SLAR [Kub-3M] - (MiG-25, NATO Name?) Radar   

                       Sensor 1 Type Sensor 2 Max Range  \
0  SLAR, Side-Looking Airborne Radar           222.2 km   

                               Sensor 2 Name                Sensor 2 Type  \
0  SPO-3 Sirena-3 - (MiG-25, NATO Name?) ESM  RWR, Radar Warning Receiver   

  Sensor 3 Max Range                                      Sensor 3 Name  \
0             9.3 km  Generic Visual Reconnaissance Camera - (4x Mag...   

                         Sensor 3 Type Sensor 4 Max Range  \
0  Visual, Reconnaissance Frame Camera             926 km   

                                Sensor 4 Name Sensor 4 Type  
0  SRS-4B Romb - (MiG-25RB, Izdeliyie 30) ESM         ELINT  
DEBUG:root:Table 1, 2 and 3 combined successfully.
DEBUG:root:table_number is 2
DEBUG:root:table_header

DEBUG:root:Table 1, 2 and 3 combined successfully.
DEBUG:root:table_number is 2
DEBUG:root:table_header_dict is {'index': [0, 1, 2], 'header': [['General data:', ''], ['Sensors / EW:'], ['Weapons / Loadouts:']]}
DEBUG:root:_index is 2
DEBUG:root:_header is Weapons / Loadouts:
DEBUG:root:i:0
DEBUG:root:_list:[[' AA-11 Archer [R-73] -', '(1986) Guided Weapon', 'Air Max: 18.5 km. '], ['Air Max: 18.5 km']]
DEBUG:root:_sublist_2:['Air Max: 18.5 km']
DEBUG:root:weapon_keys: ['Weapon 1']
DEBUG:root:weapon_values: ['Air Max: 18.5 km']
DEBUG:root:dict_keys: ['Weapon 1 Name', 'Weapon 1 Description']
DEBUG:root:dict_values: [' AA-11 Archer [R-73] -', '(1986) Guided Weapon', 'Air Max: 18.5 km. ']
DEBUG:root:i:1
DEBUG:root:_list:[[' AA-11 Archer [R-73] -', '(1986) Guided Weapon', 'Air Max: 18.5 km. '], ['Air Max: 18.5 km'], [' 1500 liter Drop Tank -', ' Drop Tank\n'], [' Drop Tank\n']]
DEBUG:root:_sublist_2:[' Drop Tank\n']
DEBUG:root:weapon_keys: ['Weapon 2']
DEBUG:root:weapon_values: [' Drop Tank

DEBUG:root:_sublist_2:['Surface Max: 1.9 km', 'Land Max: 1.9 km']
DEBUG:root:weapon_keys: ['Weapon 8 Surface Max']
DEBUG:root:weapon_values: [' 1.9 km']
DEBUG:root:weapon_keys: ['Weapon 8 Surface Max', 'Weapon 8 Land Max']
DEBUG:root:weapon_values: [' 1.9 km', ' 1.9 km']
DEBUG:root:dict_keys: ['Weapon 1 Name', 'Weapon 1 Description', 'Weapon 2 Name', 'Weapon 2 Description', 'Weapon 3 Name', 'Weapon 3 Description', 'Weapon 4 Name', 'Weapon 4 Description', 'Weapon 4 Surface Max', 'Weapon 4 Land Max', 'Weapon 5 Name', 'Weapon 5 Description', 'Weapon 5 Surface Max', 'Weapon 5 Land Max', 'Weapon 6 Name', 'Weapon 6 Description', 'Weapon 6 Surface Max', 'Weapon 6 Land Max', 'Weapon 7 Name', 'Weapon 7 Description', 'Weapon 7 Surface Max', 'Weapon 7 Land Max', 'Weapon 8 Name', 'Weapon 8 Description', 'Weapon 8 Surface Max', 'Weapon 8 Land Max']
DEBUG:root:dict_values: [' AA-11 Archer [R-73] -', '(1986) Guided Weapon', 'Air Max: 18.5 km. ', 'Air Max: 18.5 km', ' 1500 liter Drop Tank -', ' Drop T

DEBUG:root:weapon_keys: ['Weapon 11 Surface Max']
DEBUG:root:weapon_values: [' 1.9 km']
DEBUG:root:weapon_keys: ['Weapon 11 Surface Max', 'Weapon 11 Land Max']
DEBUG:root:weapon_values: [' 1.9 km', ' 1.9 km']
DEBUG:root:dict_keys: ['Weapon 1 Name', 'Weapon 1 Description', 'Weapon 2 Name', 'Weapon 2 Description', 'Weapon 3 Name', 'Weapon 3 Description', 'Weapon 4 Name', 'Weapon 4 Description', 'Weapon 4 Surface Max', 'Weapon 4 Land Max', 'Weapon 5 Name', 'Weapon 5 Description', 'Weapon 5 Surface Max', 'Weapon 5 Land Max', 'Weapon 6 Name', 'Weapon 6 Description', 'Weapon 6 Surface Max', 'Weapon 6 Land Max', 'Weapon 7 Name', 'Weapon 7 Description', 'Weapon 7 Surface Max', 'Weapon 7 Land Max', 'Weapon 8 Name', 'Weapon 8 Description', 'Weapon 8 Surface Max', 'Weapon 8 Land Max', 'Weapon 9 Name', 'Weapon 9 Description', 'Weapon 9 Surface Max', 'Weapon 9 Land Max', 'Weapon 10 Name', 'Weapon 10 Description', 'Weapon 10 Surface Max', 'Weapon 10 Land Max', 'Weapon 11 Name', 'Weapon 11 Descriptio